In [28]:
import os
from glob import glob

import pandas as pd
import numpy as np
from collections import Counter

import pickle
import gc

from tqdm.notebook import tqdm
#tqdm.pandas()

In [29]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, make_scorer

In [30]:
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import xgboost as xgb
import catboost as cb
from catboost import CatBoostClassifier

In [31]:
DATA = os.path.join('.', 'data')
DATA_OWN = os.path.join('.', 'data_own')
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = os.path.join('.', 'models')
UTILS = os.path.join('.', 'utils')
SUBM = os.path.join('.', 'submissions')

# load data / submit / features

In [32]:
load_dtypes = pickle.load(open(os.path.join(UTILS, 'load_dtypes.pkl'), 'rb'))

In [33]:
using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))

In [34]:
#data = pd.read_csv(os.path.join(DATA_OWN, 'data_pred.csv'), parse_dates=['timestamp'])
data = pd.read_csv(os.path.join(DATA_OWN, 'data_pred.csv'),  usecols=using_features, dtype=load_dtypes)
data.head()

,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,client_freq_chat,client_freq_card_recharge,client_freq_invest,...,relations_time_past_targ_main_screen,relations_time_past_targ_statement,relations_time_past_targ_credit_info,relations_time_past_targ_own_transfer,relations_time_past_targ_mobile_recharge,relations_time_past_targ_phone_money_transfer,relations_time_past_targ_card2card_transfer,relations_time_past_targ_chat,relations_time_past_targ_card_recharge,relations_time_past_targ_invest
0,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,...,0.000000,0.000000,0.550736,0.000000,0.000000,0.000000,0.000000,0.000000,1.803765,0.0
1,0.193548,0.580645,0.032258,0.032258,0.064516,0.064516,0.000000,0.032258,0.0,0.0,...,0.719404,5.552509,3.139549,3.248684,4.212135,3.773859,0.000000,0.839439,0.000000,0.0
2,0.181818,0.363636,0.272727,0.090909,0.000000,0.000000,0.000000,0.090909,0.0,0.0,...,0.006159,2.391634,0.414321,2.391750,0.000000,0.000000,0.000000,2.371550,0.000000,0.0
3,0.000000,0.777778,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.0,...,0.000000,0.522153,2.344850,0.000000,0.000000,0.000000,1.771654,0.000000,0.000000,0.0
4,0.395833,0.187500,0.000000,0.000000,0.312500,0.000000,0.000000,0.104167,0.0,0.0,...,0.041277,0.721598,0.000000,0.000000,0.760100,0.000000,0.000000,0.024920,0.000000,0.0


In [35]:
if data.isnull().values.any() == True:
    print('I have a Bad news for you!')
    data.fillna(0, inplace = True)

In [36]:
subm = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_sample_prediction.csv'))
subm.head()

,client_pin,prediction
0,f0c674b2bb4dc64be607029271d706ec,credit_info
1,90725b54ce77576883813d87749df6bd,credit_info
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,own_transfer
3,831bf4c0ecccc55e536b8cfb9153d672,statement
4,3f1a5a1c492ce877af833113e59c5797,invest


# load models

In [41]:
#clf_sgd = pickle.load(open(os.path.join(MODELS, 'clf_sgd.pkl'), 'rb'))
#clf_mlp = pickle.load(open(os.path.join(MODELS, 'clf_mlp.pkl'), 'rb'))
#clf_lr = pickle.load(open(os.path.join(MODELS, 'clf_lr.pkl'), 'rb'))
#clf_lrsvc = pickle.load(open(os.path.join(MODELS, 'clf_lrsvc.pkl'), 'rb'))
#clf_svc = pickle.load(os.paht.join(MODELS, 'clf_svc.pkl'))
#clf_lgbm = pickle.load(open(os.path.join(MODELS, 'clf_lgbm.pkl'), 'rb'))



In [16]:
clf_xgb = xgb.Booster()
clf_xgb.load_model(os.path.join(MODELS, 'clf_xgb.json'))
target_encoder = pickle.load(open(os.path.join(UTILS, 'oe_target.pkl'), 'rb'))

In [39]:
clf_cb = CatBoostClassifier()
clf_cb  = clf_cb.load_model(os.path.join(MODELS, 'clf_cb.cbm'), format='cbm')

# make predictions

In [11]:
#pred_sgd = clf_sgd.predict(data[using_features])

In [13]:
pred_mlp = clf_mlp.predict(data[using_features])

In [13]:
#pred_svc = clf_svc.predict(data[using_features])

In [12]:
#pred_lr = clf_lr.predict(data[using_features])

In [13]:
#pred_lrsvc = clf_lrsvc.predict(data[using_features])

In [26]:
#pred_lgbm = clf_lgbm.predict(data[using_features])

In [42]:
pred_cb = clf_cb.predict(data[using_features])

In [18]:
pred_xgb_int = clf_xgb.predict(xgb.DMatrix(data[using_features]))

In [24]:
pred_xgb = target_encoder.inverse_transform(pred_xgb_int.reshape(-1, 1))

# make submit

In [23]:
#subm.prediction = pred_stack

In [43]:
#subm.prediction = pred_sgd
#subm.prediction = pred_mlp
#subm.prediction = pred_lr
subm.prediction = pred_cb
#subm.prediction = pred_lrsvc
#subm.prediction = pred_lgbm
#subm.prediction = pred_xgb

In [46]:
subm.to_csv(os.path.join(SUBM, 'subm_client_6diff_lt_relat_cb.csv'), index = False)